# LDA

## Sklearn

In [1]:
from sklearn.datasets import fetch_20newsgroups

categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space',
             'comp.sys.ibm.pc.hardware', 'sci.crypt']

newsgroups_train = fetch_20newsgroups(subset = 'train', categories = categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Selected categories:', newsgroups_train.target_names)

#Train set size: 3219
#Selected categories: ['alt.atheism', 'comp.graphics', 'comp.sys.ibm.pc.hardware', 'sci.crypt', 'sci.space', 'talk.religion.misc']


In [3]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(token_pattern = "[\w']{3,}", stop_words = 'english',
                    max_features = 2000, min_df = 5, max_df = 0.5)
review_cv = cv.fit_transform(newsgroups_train.data)

- topic_word_prior (𝛽): 토픽의 사전 단어 분포
    - 보통 0.1
- doc_topic_prior (𝛼): 문서의 사전 토픽 분포
    - 보통 50/n_components
- learning_method
    - batch: 성능 우수, 느림
    - online

In [8]:
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
np.set_printoptions(precision = 3)

lda = LatentDirichletAllocation(n_components = 10,
                               max_iter = 5,
                               topic_word_prior = 0.1, doc_topic_prior = 1.0,
                               learning_method = 'online',
                               n_jobs = -1,
                               random_state = 0)

review_topics = lda.fit_transform(review_cv)
print('#shape of review_topics: ', review_topics.shape)
print('#Sample of review_topics: ', review_topics[0])

gross_topic_weights = np.mean(review_topics, axis = 0)
print('#Sum of topic weights of documents:', gross_topic_weights)
print('#shape of topic word distribution:', lda.components_.shape)

#shape of review_topics:  (3219, 10)
#Sample of review_topics:  [0.903 0.007 0.027 0.008 0.007 0.008 0.007 0.007 0.007 0.018]
#Sum of topic weights of documents: [0.087 0.083 0.085 0.115 0.115 0.126 0.098 0.072 0.07  0.148]
#shape of topic word distribution: (10, 2000)


In [13]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print('Topic #%d: ' %topic_idx, end = '')
        print(", ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
        # 위 slicing에서 맨 뒤 -1은 역순을 의미, 역순으로 했을 때 처음부터 n_top_words까지
    print()

print_top_words(lda,cv.get_feature_names_out(), 10)

Topic #0: com, morality, keith, article, sgi, think, sandvik, objective, caltech, moral
Topic #1: image, file, graphics, files, ftp, available, software, use, data, mail
Topic #2: space, nasa, access, launch, earth, orbit, shuttle, digex, lunar, satellite
Topic #3: article, com, just, don't, like, i'm, nntp, university, host, posting
Topic #4: key, clipper, chip, encryption, com, government, law, keys, use, escrow
Topic #5: scsi, com, bit, ibm, bus, know, windows, thanks, card, university
Topic #6: host, gov, nntp, posting, university, distribution, nasa, ___, world, com
Topic #7: drive, com, disk, hard, controller, drives, dos, tape, floppy, problem
Topic #8: key, public, message, faq, mail, pgp, des, group, uni, ripem
Topic #9: god, people, don't, jesus, believe, just, does, say, think, know

